<table style="width:100%" align="right">
  <tr>
    <td><center style="font-size:300%;">Transfer Learning</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>

Msc. Cristian Muñoz V.

In [0]:
from keras.models import Model,Input
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.applications import  VGG16
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.optimizers import SGD
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from keras import losses
from keras.optimizers import RMSprop

import cv2
import numpy as np
from PIL import Image
from IPython.display import Image
from random import randint
import matplotlib.pylab as plt
from transfer_learning_utils import *

import datetime
from dataset_utils import *
now = datetime.datetime.now

# Abrir Modelo Pretreinado

*Modelo*

In [0]:
pretrained_model = VGG16(weights="imagenet", include_top=True)

for i, layer in enumerate(pretrained_model.layers):
    print(i , layer.name, layer.output_shape)

Escolher a camada adecuada para realizar transfer Learning


*Modelo*

In [0]:
bottleneck_model=Model(inputs  = pretrained_model.input,
                       outputs = pretrained_model.get_layer('flatten').output)
bottleneck_model.summary()

# Dataset yalefaces

In [ ]:
imagem=load_img('img/subject.jpg', target_size=(224,224,3));imagem

In [ ]:
Image("img/Alldataset.png", width=800, height=300)
img_array=img_to_array(imagem).astype('float32')
img_array=img_array.reshape((1,224,224,3))
x=preprocess_input(img_array)
features=bottleneck_model.predict(x)
features.shape

Abrir o dataset :

In [0]:
(X_train,y_train) , (X_test,y_test) = load_data('yalefaces', 0.2)
print('train samples:{}'.format(X_train.shape[0]))
print('test samples:{}'.format(X_test.shape[0]))

Realizamos o processo de data augmentation para os dados de treinamento e teste

In [0]:
train_datagen = ImageDataGenerator(rescale=1,
                               shear_range=0.2, 
                               zoom_range=0.2, 
                               horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1)

batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, 
                                     batch_size=batch_size, 
                                     shuffle=False)

test_generator = test_datagen.flow(X_test, y_test, 
                                     batch_size=batch_size, 
                                     shuffle=False)

Geração de features

In [0]:
train_steps_per_epoch = X_train.shape[0]//batch_size + 1
test_steps_per_epoch = X_test.shape[0]//batch_size + 1

increase_by=1
steps_per_epoch = train_steps_per_epoch*increase_by
train_features=[]
train_labels=[]
for i in range(steps_per_epoch):
    print("\rTraining Batches: {}/{}".format(i+1,steps_per_epoch), end="")
    x,y = next(train_generator)
    x=preprocess_input(x)
    train_features.append(bottleneck_model.predict(x))
    train_labels.append(y)

test_features=[]
test_labels=[]
for i in range(test_steps_per_epoch):
    print("\rTest Batches: {}/{}".format(i+1,test_steps_per_epoch), end="")
    x,y = next(test_generator)
    x=preprocess_input(x)
    test_features.append(bottleneck_model.predict(x))
    test_labels.append(y)

train_features = np.concatenate(train_features,axis=0)
train_labels = np.concatenate(train_labels,axis=0)
test_features = np.concatenate(test_features,axis=0)
test_labels = np.concatenate(test_labels,axis=0)

Como a extração de features é um processo demorado, se recomenda salvar  os features casso seja necessario restar o kernel.

In [0]:
train_data = (train_features,train_labels)
test_data = (test_features,test_labels)
pickle.dump(train_data, open('train_features.p','wb'))
pickle.dump(test_data, open('test_features.p','wb'))

In [0]:
(train_features,train_labels) = pickle.load( open( "train_features.p", "rb" ) )
(test_features,test_labels) = pickle.load( open( "test_features.p", "rb" ))

Definimos nosso classificador:

In [0]:
inputs = Input(bottleneck_model.get_layer('flatten').output_shape[1:])
x = Dense(128)(inputs)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(15)(x)
x = Activation('softmax')(x)
model_cls = Model(inputs=inputs, outputs=x)
model_cls.summary()

Treinamos o modelo

In [0]:
batch_size=32
epochs=30
from keras import optimizers
# Agora, vamos treinar nosso modelo nos dígitos 0,1,2,3,4
model_cls.compile(loss='categorical_crossentropy',
              optimizer='adam',#sgd,
              metrics=['accuracy'])

t = now()
history=model_cls.fit(train_features, train_labels,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(test_features, test_labels))

print('Training time: %s' % (now() - t))
scorelt5 = model_cls.evaluate(test_features, test_labels, verbose=0)
print('Test score:', scorelt5[0])
print('Test accuracy:', scorelt5[1])